In [ ]:
import pandas as pd
import json
import os
import sys
import re
import time
import warnings
from openpyxl import load_workbook
warnings.filterwarnings('ignore')

import requests
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

YOUTUBE_COMMENTS_AJAX_URL = 'https://www.youtube.com/comment_service_ajax'

USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'
FILE_NAME = 'finalcsv.csv'
FILE_NAMEx = 'finalxlsx.xlsx'
COMMENT_LIMIT = 100000
YT_CFG_RE = r'ytcfg\.set\s*\(\s*({.+?})\s*\)\s*;'
YT_INITIAL_DATA_RE = r'(?:window\s*\[\s*["\']ytInitialData["\']\s*\]|ytInitialData)\s*=\s*({.+?})\s*;\s*(?:var\s+meta|</script|\n)'


def regex_search(text, pattern, group=1, default=None):
    match = re.search(pattern, text)
    return match.group(group) if match else default


def ajax_request(session, endpoint, ytcfg, retries=5, sleep=20):
    url = 'https://www.youtube.com' + endpoint['commandMetadata']['webCommandMetadata']['apiUrl']

    data = {'context': ytcfg['INNERTUBE_CONTEXT'],
            'continuation': endpoint['continuationCommand']['token'],
            'content-length': '3201'}

    for _ in range(retries):
        response = session.post(url, params={'key': ytcfg['INNERTUBE_API_KEY']}, json=data)
        if response.status_code == 200:
            return response.json()
        if response.status_code in [403, 413]:
            return {}
        else:
            time.sleep(sleep)

def download_comments(YOUTUBE_VIDEO_URL, sort_by=SORT_BY_RECENT, language=None, sleep=0.1):
    session = requests.Session()
    session.headers['User-Agent'] = USER_AGENT
    response = session.get(YOUTUBE_VIDEO_URL)

    if 'uxe=' in response.request.url:
        session.cookies.set('CONSENT', 'YES+cb', domain='.youtube.com')
        response = session.get(YOUTUBE_VIDEO_URL)

    html = response.text
    ytcfg = json.loads(regex_search(html, YT_CFG_RE, default=''))
    if not ytcfg:
        return  
    if language:
        ytcfg['INNERTUBE_CONTEXT']['client']['hl'] = language

    data = json.loads(regex_search(html, YT_INITIAL_DATA_RE, default=''))
    
    section = next(search_dict(data, 'itemSectionRenderer'), None)
    renderer = next(search_dict(section, 'continuationItemRenderer'), None) if section else None
    if not renderer:
        return

    needs_sorting = sort_by != SORT_BY_POPULAR
    continuations = [renderer['continuationEndpoint']]
    while continuations:
        continuation = continuations.pop()
        response = ajax_request(session, continuation, ytcfg)
        if not response:
            break
        if list(search_dict(response, 'externalErrorMessage')):
            raise RuntimeError('Error returned from server: ' + next(search_dict(response, 'externalErrorMessage')))

        actions = list(search_dict(response, 'reloadContinuationItemsCommand')) + \
                  list(search_dict(response, 'appendContinuationItemsAction'))
        for action in actions:
            for item in action.get('continuationItems', []):
                if action['targetId'] == 'comments-section':
                    continuations[:0] = [ep for ep in search_dict(item, 'continuationEndpoint')]
                if action['targetId'].startswith('comment-replies-item') and 'continuationItemRenderer' in item:
                    continuations.append(next(search_dict(item, 'buttonRenderer'))['command'])

        for comment in reversed(list(search_dict(response, 'commentRenderer'))):
            yield {'text': ''.join([c['text'] for c in comment['contentText'].get('runs', [])]),
                  'votes': comment.get('voteCount', {}).get('simpleText', '0')}

        time.sleep(sleep)


def search_dict(partial, search_key):
    stack = [partial]
    while stack:
        current_item = stack.pop()
        if isinstance(current_item, dict):
            for key, value in current_item.items():
                if key == search_key:
                    yield value
                else:
                    stack.append(value)
        elif isinstance(current_item, list):
            for value in current_item:
                stack.append(value)


def main(url):
    df_comment = pd.DataFrame()
    try:
        youtube_url = url
        limit = COMMENT_LIMIT
        val = {"video_id" : '', "comment_1" : '', "comment_1_likes" : '', "comment_2" : '', "comment_2_likes" : '',
               "comment_3" : '', "comment_3_likes" : '', "comment_4" : '', "comment_4_likes" : '', 
               "comment_5" : '', "comment_5_likes" : '', "comment_6" : '', "comment_6_likes" : '', 
               "comment_7" : '', "comment_7_likes" : '', "comment_8" : '', "comment_8_likes" : '', 
               "comment_9" : '', "comment_9_likes" : '', "comment_10" : '', "comment_10_likes" : '', 
               "comment_11" : '', "comment_11_likes" : '', "comment_12" : '', "comment_12_likes" : '',
               "comment_13" : '', "comment_13_likes" : '', "comment_14" : '', "comment_14_likes" : '', 
               "comment_15" : '', "comment_15_likes" : '', "comment_16" : '', "comment_16_likes" : '',
               "comment_17" : '', "comment_17_likes" : '', "comment_18" : '', "comment_18_likes" : '',
               "comment_19" : '', "comment_19_likes" : '', "comment_20" : '', "comment_20_likes" : ''}

        count = 0
        votes = 0
        start_time = time.time()
        val["video_id"] = youtube_url.replace("https://www.youtube.com/watch?v=","")

        for comment in download_comments(youtube_url):

            count += 1
            votes = comment["votes"]
            if("M" in votes or "m" in votes):
                votes = votes.replace("M","")
                votes = votes.replace("m","")
                pp = float(votes)
                votes =  pp * 1000000
            elif("K" in votes or "k" in votes):
                votes = votes.replace("k","")
                votes = votes.replace("K","")
                pp = float(votes)
                votes =  pp * 1000
            val["comment_"+str(count)] = comment["text"]
            val["comment_"+str(count)+"_likes"] = int(votes)
        print(url)
        if len(val["video_id"]) == 11:
            df_comment = df_comment.append(val, ignore_index=True)
            if not os.path.isfile(FILE_NAME):
                df_comment.to_csv(FILE_NAME, encoding='utf-8', index=False)
            else:  
                df_comment.to_csv(FILE_NAME, mode='a', encoding='utf-8', index=False, header=False)
            if not os.path.isfile(FILE_NAMEx):
                df_comment.to_excel(FILE_NAMEx, encoding='utf-8', index=False)
            else:  
                book = load_workbook(FILE_NAMEx)
                writer = pd.ExcelWriter(FILE_NAMEx, engine='openpyxl')
                writer.book = book
                print(writer.sheets["Sheet1"].max_row)
                df_comment.to_excel(writer,sheet_name="Sheet1", startrow=writer.sheets["Sheet1"].max_row,encoding='utf-8', index = False,header= False)
                writer.save()
            print('[{:.2f} seconds] Done!'.format(time.time() - start_time))

    except Exception as e:
        print('Error:', str(e))
        sys.exit(1)


df_video_list = pd.read_excel('video_li.xlsx')
print(df_video_list['link'].map(lambda x: main(x)))


https://www.youtube.com/watch?v=3cxtvDEziXU
2764
[5.64 seconds] Done!
https://www.youtube.com/watch?v=3d-6YanrbL4
2765
[6.07 seconds] Done!
https://www.youtube.com/watch?v=3d-zGTXqBqI
2766
[6.17 seconds] Done!
https://www.youtube.com/watch?v=3d1JjfP8Dno
2767
[6.12 seconds] Done!
https://www.youtube.com/watch?v=3d2D4akj5_8
2768
[5.98 seconds] Done!
https://www.youtube.com/watch?v=3d4Zq-IQsuA
2769
[5.99 seconds] Done!
https://www.youtube.com/watch?v=3dKgPKuQt4I
2770
[5.85 seconds] Done!
https://www.youtube.com/watch?v=3dRHSvdRpzg
2771
[6.27 seconds] Done!
https://www.youtube.com/watch?v=3dWvR_RXPhQ
2772
[5.69 seconds] Done!
https://www.youtube.com/watch?v=3dZJ0u2cHVA
2773
[5.34 seconds] Done!
https://www.youtube.com/watch?v=3daTG4_JS_4
2774
[5.89 seconds] Done!
https://www.youtube.com/watch?v=3dazyiv2h2Q
2775
[5.90 seconds] Done!
https://www.youtube.com/watch?v=3ddh1nNOaS8
2776
[6.24 seconds] Done!
https://www.youtube.com/watch?v=3de64yKeElM
2777
[6.15 seconds] Done!
https://www.youtube.

2881
[5.88 seconds] Done!
https://www.youtube.com/watch?v=3niVHCdB1wA
2882
[6.17 seconds] Done!
https://www.youtube.com/watch?v=3nskU3BoGtc
2883
[6.33 seconds] Done!
https://www.youtube.com/watch?v=3oBCtTv6yOw
2884
[6.21 seconds] Done!
https://www.youtube.com/watch?v=3oCeU0HthEo
2885
[6.19 seconds] Done!
https://www.youtube.com/watch?v=3oNQpM8Twh8
2886
[5.98 seconds] Done!
https://www.youtube.com/watch?v=3oUwf3HNPyo
2887
[5.99 seconds] Done!
https://www.youtube.com/watch?v=3oZ6GDYGA6Q
2888
[6.24 seconds] Done!
https://www.youtube.com/watch?v=3oaf7tLz6O4
2889
[6.71 seconds] Done!
https://www.youtube.com/watch?v=3oaj8Dlscco
2890
[6.07 seconds] Done!
https://www.youtube.com/watch?v=3od-kQMTZ9M
2891
[6.12 seconds] Done!
https://www.youtube.com/watch?v=3oe8ToTjlQM
2892
[6.26 seconds] Done!
https://www.youtube.com/watch?v=3okShyDl_UY
2893
[6.21 seconds] Done!
https://www.youtube.com/watch?v=3olsPmfbkVU
2894
[6.14 seconds] Done!
https://www.youtube.com/watch?v=3ourbGSpPHE
2895
[6.08 seconds] 

[6.44 seconds] Done!
https://www.youtube.com/watch?v=3zFS-V_iCiM
2999
[6.41 seconds] Done!
https://www.youtube.com/watch?v=3zPr7olHUu8
3000
[6.68 seconds] Done!
https://www.youtube.com/watch?v=3zXERomRwNE
3001
[6.20 seconds] Done!
https://www.youtube.com/watch?v=3zcdViww_vI
3002
[6.50 seconds] Done!
https://www.youtube.com/watch?v=3zsPWw2H9PE
3003
[6.27 seconds] Done!
https://www.youtube.com/watch?v=4--QUp7fVuU
3004
[6.19 seconds] Done!
https://www.youtube.com/watch?v=4-0uQnaO_U0
3005
[6.57 seconds] Done!
https://www.youtube.com/watch?v=4-V7spSg1CQ
3006
[6.15 seconds] Done!
https://www.youtube.com/watch?v=4-dCVUL7zoM
3007
[6.13 seconds] Done!
https://www.youtube.com/watch?v=4-keAPArp9A
3008
[6.11 seconds] Done!
https://www.youtube.com/watch?v=4-p2S5q0Qrw
3009
[6.28 seconds] Done!
https://www.youtube.com/watch?v=4-r1bM5YTQI
3010
[6.15 seconds] Done!
https://www.youtube.com/watch?v=4-xLIIqBG9Y
3011
[5.97 seconds] Done!
https://www.youtube.com/watch?v=4-z6imVeNF8
3012
[6.50 seconds] Done!

https://www.youtube.com/watch?v=4BuIRH78LwI
3116
[6.57 seconds] Done!
https://www.youtube.com/watch?v=4BvecJFBzeg
3117
[6.49 seconds] Done!
https://www.youtube.com/watch?v=4CFRY6ORN8I
3118
[6.59 seconds] Done!
https://www.youtube.com/watch?v=4CLCbO3E-wE
3119
[5.44 seconds] Done!
https://www.youtube.com/watch?v=4CUcu5FcCFU
3120
[7.05 seconds] Done!
https://www.youtube.com/watch?v=4Cf_s21NXVA
3121
[6.69 seconds] Done!
https://www.youtube.com/watch?v=4Cip_vZwFQY
3122
[6.48 seconds] Done!
https://www.youtube.com/watch?v=4CqwkiaXEOQ
3123
[6.77 seconds] Done!
https://www.youtube.com/watch?v=4CvuPz6AO3I
3124
[6.42 seconds] Done!
https://www.youtube.com/watch?v=4D9_UnbQ6CU
3125
[6.44 seconds] Done!
https://www.youtube.com/watch?v=4DW6Ypr1L6s
3126
[6.55 seconds] Done!
https://www.youtube.com/watch?v=4DWkX_oVRlY
3127
[6.67 seconds] Done!
https://www.youtube.com/watch?v=4DdPBhB2BOQ
3128
[6.58 seconds] Done!
https://www.youtube.com/watch?v=4DfCXjwQBRA
3129
[7.08 seconds] Done!
https://www.youtube.

3233
[6.97 seconds] Done!
https://www.youtube.com/watch?v=4OiBBB8SxmM
3234
[6.60 seconds] Done!
https://www.youtube.com/watch?v=4OnTMAEvQMY
3235
[6.84 seconds] Done!
https://www.youtube.com/watch?v=4OroVLkVbhc
3236
[6.87 seconds] Done!
https://www.youtube.com/watch?v=4OtIUIjx0s4
3237
[7.13 seconds] Done!
https://www.youtube.com/watch?v=4Oy9s3aHpjo
3238
[6.76 seconds] Done!
https://www.youtube.com/watch?v=4OzUtR54ou4
3239
[6.62 seconds] Done!
https://www.youtube.com/watch?v=4P4TNAG_DqY
3240
[6.75 seconds] Done!
https://www.youtube.com/watch?v=4P5toaChPeg
3241
[6.67 seconds] Done!
https://www.youtube.com/watch?v=4P8fKd0IVOs
3242
[7.23 seconds] Done!
https://www.youtube.com/watch?v=4P9K55yPcBw
3243
[6.55 seconds] Done!
https://www.youtube.com/watch?v=4P9XUrniiK4
3244
[6.64 seconds] Done!
https://www.youtube.com/watch?v=4PAY9VNfMVg
3245
[6.57 seconds] Done!
https://www.youtube.com/watch?v=4PJpkGPtAHE
3246
[6.78 seconds] Done!
https://www.youtube.com/watch?v=4PLeqEWdP5E
3247
[5.97 seconds] 

...

https://www.youtube.com/watch?v=HI5GAXtwXcE
10811
[13.90 seconds] Done!
https://www.youtube.com/watch?v=HI7kVbXZdx0
10812
[14.07 seconds] Done!
https://www.youtube.com/watch?v=HIIAMnyf4PM
10813
[14.16 seconds] Done!
https://www.youtube.com/watch?v=HIVv5TLrf8g
10814
[14.08 seconds] Done!
https://www.youtube.com/watch?v=HInaDcbkoIg
10815
[13.61 seconds] Done!
https://www.youtube.com/watch?v=HIp5rzhsMSw
10816
[13.75 seconds] Done!
https://www.youtube.com/watch?v=HIp70O56oqo
10817
[14.71 seconds] Done!
https://www.youtube.com/watch?v=HIp_rju1P-g
10818
[13.78 seconds] Done!
https://www.youtube.com/watch?v=HIsdOfdSkwU
10819
[13.83 seconds] Done!
https://www.youtube.com/watch?v=HIv3ClkEhrA
10820
[14.06 seconds] Done!
https://www.youtube.com/watch?v=HJ0Jw3oY9Z8
10821
[13.40 seconds] Done!
https://www.youtube.com/watch?v=HJZVyoWS9fU
10822
[13.50 seconds] Done!
https://www.youtube.com/watch?v=HJiFaPCHn2s
10823
[13.93 seconds] Done!
https://www.youtube.com/watch?v=HJnEl2WShWc
10824
[13.78 seconds

https://www.youtube.com/watch?v=HVR7u48De2U
10925
[16.74 seconds] Done!
https://www.youtube.com/watch?v=HVcbcDsepag
10926
[14.03 seconds] Done!
https://www.youtube.com/watch?v=HVczpKy0MP4
10927
[14.10 seconds] Done!
https://www.youtube.com/watch?v=HVfqR522kAU
10928
[12.91 seconds] Done!
https://www.youtube.com/watch?v=HVlyzs5qjII
10929
[13.86 seconds] Done!
https://www.youtube.com/watch?v=HVq-o5dL6jk
10930
[13.73 seconds] Done!
https://www.youtube.com/watch?v=HVt2AyPP6Jc
10931
[14.56 seconds] Done!
https://www.youtube.com/watch?v=HVxgGY39MaY
10932
[14.27 seconds] Done!
https://www.youtube.com/watch?v=HW8tKKPDqD8
10933
[14.70 seconds] Done!
https://www.youtube.com/watch?v=HWB17i3VyOY
10934
[16.39 seconds] Done!
https://www.youtube.com/watch?v=HWB5snVp7JA
10935
[15.04 seconds] Done!
https://www.youtube.com/watch?v=HWPPonnqnis
10936
[16.02 seconds] Done!
https://www.youtube.com/watch?v=HWi4sRXO5LI
10937
[14.39 seconds] Done!
https://www.youtube.com/watch?v=HX-R25i25rU
10938
[14.17 seconds

https://www.youtube.com/watch?v=HhQ6HzQKNyw
11039
[15.68 seconds] Done!
https://www.youtube.com/watch?v=HhU07nmKAMk
11040
[16.03 seconds] Done!
https://www.youtube.com/watch?v=HhkgdTxbpRA
11041
[15.34 seconds] Done!
https://www.youtube.com/watch?v=HhmpAHccwp0
11042
[14.96 seconds] Done!
https://www.youtube.com/watch?v=HhnT-U-uAlA
11043
[15.58 seconds] Done!
https://www.youtube.com/watch?v=Hi0zbU3DW6o
11044
[15.55 seconds] Done!
https://www.youtube.com/watch?v=Hi1IyF9IiXQ
11045
[16.32 seconds] Done!
https://www.youtube.com/watch?v=Hi2gayxePM4
11046
[15.59 seconds] Done!
https://www.youtube.com/watch?v=Hi4ktzK9g0I
11047
[15.47 seconds] Done!
https://www.youtube.com/watch?v=Hi53L3jEnA8
11048
[16.30 seconds] Done!
https://www.youtube.com/watch?v=HiBwVq10Eno
11049
[15.07 seconds] Done!
https://www.youtube.com/watch?v=HiFMBLFzk_U
11050
[15.20 seconds] Done!
https://www.youtube.com/watch?v=HiITxbXAzCg
11051
[14.89 seconds] Done!
https://www.youtube.com/watch?v=HiPkwl5p1GY
11052
[16.38 seconds

https://www.youtube.com/watch?v=Huh2A8x8IxQ
11153
[14.61 seconds] Done!
https://www.youtube.com/watch?v=HuhahJVnhfw
11154
[14.20 seconds] Done!
https://www.youtube.com/watch?v=Hul56IzA1V8
11155
[14.52 seconds] Done!
https://www.youtube.com/watch?v=HulxxPkk6DQ
11156
[14.57 seconds] Done!
https://www.youtube.com/watch?v=Hup7TxTDkSI
11157
[14.51 seconds] Done!
https://www.youtube.com/watch?v=Huphf4_tvMU
11158
[13.82 seconds] Done!
https://www.youtube.com/watch?v=HvAkAqEkZXM
11159
[14.60 seconds] Done!
https://www.youtube.com/watch?v=HvVkNc71n-Y
11160
[14.66 seconds] Done!
https://www.youtube.com/watch?v=HvYG-_dli3w
11161
[14.85 seconds] Done!
https://www.youtube.com/watch?v=HvjVdlMqwH4
11162
[14.06 seconds] Done!
https://www.youtube.com/watch?v=HvmDxl4YTW0
11163
[14.53 seconds] Done!
https://www.youtube.com/watch?v=HvoInQ86liQ
11164
[14.59 seconds] Done!
https://www.youtube.com/watch?v=HvqgpU0qxgc
11165
[14.89 seconds] Done!
https://www.youtube.com/watch?v=HvxsQq_wqWw
11166
[14.61 seconds

https://www.youtube.com/watch?v=I4Yr1Ulu9L8
11267
[16.00 seconds] Done!
https://www.youtube.com/watch?v=I4_FAOdYb-A
11268
[16.08 seconds] Done!
https://www.youtube.com/watch?v=I4yJ8KKGfpY
11269
[15.56 seconds] Done!
https://www.youtube.com/watch?v=I54yV3eVBCI
11270
[15.47 seconds] Done!
https://www.youtube.com/watch?v=I58Wj9toXUc
11271
[15.82 seconds] Done!
https://www.youtube.com/watch?v=I5HfLtEatHI
11272
[16.03 seconds] Done!
https://www.youtube.com/watch?v=I5Jr_smeIqk
11273
[15.40 seconds] Done!
https://www.youtube.com/watch?v=I5PB2JvLGro
11274
[15.45 seconds] Done!
https://www.youtube.com/watch?v=I5XC1swFQTE
11275
[16.42 seconds] Done!
https://www.youtube.com/watch?v=I5avf400uAk
11276
[15.92 seconds] Done!
https://www.youtube.com/watch?v=I5b0ZPFVrmE
11277
[17.67 seconds] Done!
https://www.youtube.com/watch?v=I5jZWu5KG9w
11278
[16.81 seconds] Done!
https://www.youtube.com/watch?v=I5rXiktth44
11279
[16.00 seconds] Done!
https://www.youtube.com/watch?v=I5rqu_xD4ho
11280
[18.73 seconds